<a href="https://colab.research.google.com/github/fanzipei/scraping_japanese_weather_station/blob/master/scraping_weather_station_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import bs4
import requests

In [23]:
from urllib.parse import urlparse, parse_qs

In [84]:
import time

In [87]:
import folium as fl

In [115]:
import pandas as pd

In [5]:
r = requests.get('http://www.data.jma.go.jp/obd/stats/etrn/select/prefecture00.php?prec_no=&block_no=&year=&month=&day=&view=')

In [6]:
bs_page = bs4.BeautifulSoup(r.text)

In [11]:
pref_url_list = []

for pref_info in bs_page.select_one('map').select('area'):
  pref_url_list.append('http://www.data.jma.go.jp/obd/stats/etrn/select/' + pref_info.attrs['href'])

In [81]:
def get_station_info_list(pref_url):

  r = requests.get(pref_url)
  bs_page = bs4.BeautifulSoup(r.text.encode('ISO-8859-1').decode(requests.utils.get_encodings_from_content(r.text)[0]), 'html.parser')

  station_url_set = set([])
  station_info_list = []

  for station_info in bs_page.select_one('map').select('area'):
    station_url = station_info.attrs['href']
    if 'onmouseover' in station_info.attrs and station_url not in station_url_set:
      station_url_set.add(station_url)
      station_dict = parse_qs(urlparse(station_url).query)
      lat_d, lat_m, lon_d, lon_m = map(lambda x: float(x[1:-1]), station_info.attrs['onmouseover'].split(',')[4:8])
      lat = lat_d + lat_m / 60.0
      lon = lon_d + lon_m / 60.0
      station_info_list.append((station_info.attrs['alt'], lat, lon, station_dict['prec_no'][0], station_dict['block_no'][0]))

  return station_info_list

In [85]:
station_info_list_all = []

for pref_url in pref_url_list:
  print(pref_url)
  station_info_list_all += get_station_info_list(pref_url)
  time.sleep(0.1)

http://www.data.jma.go.jp/obd/stats/etrn/select/prefecture.php?prec_no=11&block_no=&year=&month=&day=&view=
http://www.data.jma.go.jp/obd/stats/etrn/select/prefecture.php?prec_no=12&block_no=&year=&month=&day=&view=
http://www.data.jma.go.jp/obd/stats/etrn/select/prefecture.php?prec_no=13&block_no=&year=&month=&day=&view=
http://www.data.jma.go.jp/obd/stats/etrn/select/prefecture.php?prec_no=14&block_no=&year=&month=&day=&view=
http://www.data.jma.go.jp/obd/stats/etrn/select/prefecture.php?prec_no=15&block_no=&year=&month=&day=&view=
http://www.data.jma.go.jp/obd/stats/etrn/select/prefecture.php?prec_no=16&block_no=&year=&month=&day=&view=
http://www.data.jma.go.jp/obd/stats/etrn/select/prefecture.php?prec_no=17&block_no=&year=&month=&day=&view=
http://www.data.jma.go.jp/obd/stats/etrn/select/prefecture.php?prec_no=18&block_no=&year=&month=&day=&view=
http://www.data.jma.go.jp/obd/stats/etrn/select/prefecture.php?prec_no=19&block_no=&year=&month=&day=&view=
http://www.data.jma.go.jp/ob

In [88]:
fmap = fl.Map(location=(35.5, 139.5))

In [90]:
for station_name, lat, lon, _, _ in station_info_list_all:
  fl.Circle((lat, lon), 20, popup=station_name, width=-1).add_to(fmap)

In [91]:
fmap

In [130]:
import pickle as pk

In [ ]:
for d in range(1, 31):
  print('Date: 202006{:02d}'.format(d))
  station_info_extended = []
  for station_name, lat, lon, prec_no, block_no in station_info_list_all:
    station_url = 'http://www.data.jma.go.jp/obd/stats/etrn/view/10min_a1.php?prec_no=' + prec_no + '&block_no=' + block_no + '&year=2020&month=6&day=' + str(d) + '&view='
    try:
      df = pd.read_html(station_url)[0]
      print('a', station_url)
    except ValueError:
      station_url = 'http://www.data.jma.go.jp/obd/stats/etrn/view/10min_s1.php?prec_no=' + prec_no + '&block_no=' + block_no + '&year=2020&month=6&day=' + str(d) + '&view='
      print('s', station_url)
      df = pd.read_html(station_url)[0]

    station_info_extended.append((station_name, lat, lon, prec_no, block_no, df))

    time.sleep(0.4)
  
  with open('/content/drive/My Drive/data/japan_weather/202006{:02d}.pk'.format(d), 'wb') as f:
    pk.dump(station_info_extended, f)

a http://www.data.jma.go.jp/obd/stats/etrn/view/10min_a1.php?prec_no=36&block_no=0311&year=2020&month=6&day=2&view=
a http://www.data.jma.go.jp/obd/stats/etrn/view/10min_a1.php?prec_no=36&block_no=0312&year=2020&month=6&day=2&view=
s http://www.data.jma.go.jp/obd/stats/etrn/view/10min_s1.php?prec_no=36&block_no=47598&year=2020&month=6&day=2&view=
a http://www.data.jma.go.jp/obd/stats/etrn/view/10min_a1.php?prec_no=36&block_no=1031&year=2020&month=6&day=2&view=
a http://www.data.jma.go.jp/obd/stats/etrn/view/10min_a1.php?prec_no=36&block_no=1044&year=2020&month=6&day=2&view=
a http://www.data.jma.go.jp/obd/stats/etrn/view/10min_a1.php?prec_no=36&block_no=1116&year=2020&month=6&day=2&view=
a http://www.data.jma.go.jp/obd/stats/etrn/view/10min_a1.php?prec_no=36&block_no=1127&year=2020&month=6&day=2&view=
a http://www.data.jma.go.jp/obd/stats/etrn/view/10min_a1.php?prec_no=36&block_no=1130&year=2020&month=6&day=2&view=
a http://www.data.jma.go.jp/obd/stats/etrn/view/10min_a1.php?prec_no=36

In [ ]:
station_info_extended